In [1]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

import fonctions
from fonctions import scan_xml, modif_chemins_declaration, id_table

# options
pd.set_option('display.max_row', 200)
pd.set_option('display.max_columns', 50)

# dossier
folder = 'D:/Fichiers_dev/test_xml/tables/'
folder2 = 'D:/Fichiers_dev/test_xml/'

data3 = pd.read_csv(folder + 'data_nettoyees.csv')

In [ ]:
# conditions de mapping entre le chemin xml et les champs de la table en cours
mask_champs_ressources = (data3['chemins'].str.contains('mandatElectifDto/items/', na=False))

champs_ressources = data3[mask_champs_ressources]['chemins'].unique()
champs_ressources

# creation de la table
ressources = data3[data3['chemins'].isin(champs_ressources)][['id_declaration', 'champs', 'valeurs']]
ressources.head(100)

ressources.shape

# ajout de l'id_mandat
ids_mandat2 = []
champs_mandat2 = []

# creation de l'id_mandat, qui s'incrémente à chaque nouveau mandat, chaque nouveau id dans le tableau
for element in ressources['champs'].values:
    champs_mandat2.append(element)
    i = f'm{champs_mandat2.count("id") - 1}'
    ids_mandat2.append(i)

# creation de la colonne id_activite (car il y a d'autres activités que les mandats) dans ressources, en tant que clé étrangère
ressources['id_activite'] = ids_mandat2
ressources.head(100)

# suppression des colonnes qui ne sont pas des ressources
ressources = ressources[ressources['champs'].isin(['annee', 'montant'])]
ressources.head()

# ajout de l'id_ressources
ids_ressources = []
champs_ressources = []

# creation de l'id_ressources, qui s'incrémente à chaque nouvelle année
for element in ressources['champs'].values:
    champs_ressources.append(element)
    i = champs_ressources.count("annee") - 1
    ids_ressources.append(i)

# creation de la colonne id_ressources dans ressources, en tant que clé primaire
ressources['id_ressource'] = ids_ressources
ressources.head(100)

ressources = ressources.drop(ressources[ressources.duplicated(subset=['id_declaration',
                                                         'champs', 'id_activite', 'id_ressource'], keep=False)].index)

# pivot de la table ressources
ressources = ressources.pivot(index=['id_declaration', 'id_activite', 'id_ressource'],
                                          columns='champs', values='valeurs').reset_index()
ressources.head(100)

ressources.shape

# encodage id_declaration
ressources['id_declaration'] = encoder_id_declaration.transform(ressources['id_declaration'])

ressources.head()

# export en csv
ressources.to_csv(folder + 'ressources1.csv', index=False)